In [1]:
## Import Packages / Setup the matplotlib configuration

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils import tensorboard
from torch import optim, nn, utils
import pytorch_lightning as pl

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

np.random.seed(42)

/home/studio-lab-user/.conda/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Linear Regression

In [2]:
class LinearRegression(pl.LightningModule):
    
    def __init__(self, x_dim, y_dim, lr):
        super().__init__()
        self.save_hyperparameters()
        self.linear = nn.Linear(x_dim, y_dim)
        self.lr = lr
    
    def forward(self, x):
        return self.linear(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer

In [3]:
class LinearRegressionDataModule(pl.LightningDataModule):
    def __init__(self, weights, error_size, n_samples_train = 100000, n_samples_test = 50000, batch_size=32):
        super().__init__()
        x_dim, y_dim = weights.size()
        self.x_train = torch.randn(n_samples_train, x_dim)
        self.y_train = self.x_train@weights + error_size * torch.randn(n_samples_train, y_dim)
        self.x_test = torch.randn(n_samples_test, x_dim)
        self.y_test = self.x_test@weights + error_size * torch.randn(n_samples_test, y_dim)
        self.batch_size = batch_size

    def train_dataloader(self):
        dataset = torch.utils.data.TensorDataset(self.x_train, self.y_train)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size = self.batch_size, shuffle = True, num_workers = 4)
        return dataloader

In [4]:
weights = torch.Tensor([[1,2], [3,4], [5,6]])
x_dim, y_dim = weights.size()
model = LinearRegression(x_dim, y_dim, lr = 0.1)
data_module = LinearRegressionDataModule(weights, error_size = 0.1)
trainer = pl.Trainer(max_epochs = 1)
trainer.fit(model, datamodule = data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/studio-lab-user/sagemaker-studiolab-notebooks/ML_exercise/DeepLearning/lightning_logs

  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 8     
----------------------------------
8         Trainable params
0         Non-trainable params
8         Total params
0.000     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 3125/3125 [00:14<00:00, 211.35it/s, loss=0.00953, v_num=0]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3125/3125 [00:14<00:00, 211.26it/s, loss=0.00953, v_num=0]


In [5]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[0.9983, 2.9988, 4.9929],
                      [2.0001, 3.9973, 6.0060]])),
             ('linear.bias', tensor([0.0019, 0.0061]))])

In [6]:
loaded_model = LinearRegression.load_from_checkpoint("lightning_logs/version_0/checkpoints/epoch=0-step=3125.ckpt")
loaded_model.eval()
loaded_model.state_dict()

OrderedDict([('linear.weight',
              tensor([[0.9983, 2.9988, 4.9929],
                      [2.0001, 3.9973, 6.0060]])),
             ('linear.bias', tensor([0.0019, 0.0061]))])